# Imports

In [1]:
import logging
import os
from concurrent.futures import ThreadPoolExecutor

import cv2
import numpy as np
import torch
from pymongo import MongoClient
from scenedetect import VideoManager, SceneManager
from scenedetect.detectors import ContentDetector, ThresholdDetector
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input as preprocess_input_resnet
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as preprocess_input_vgg
from tqdm import tqdm


2024-05-19 16:32:16.416177: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Video Pre-processing

In [10]:
import cv2
import os
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# Configuration
input_dir = 'V3C1-100/'
output_dir = 'preprocessed_videos/'
output_format = 'mp4'
resize_width = 640
resize_height = 480
convert_to_grayscale = False
frame_rate = 24  # Target frame rate
max_workers = 1

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

def preprocess_video(input_path, output_path, resize_dim, grayscale, frame_rate):
    try:
        print(f"Processing video: {input_path}")
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            raise ValueError(f"Failed to open video file: {input_path}")

        original_frame_rate = cap.get(cv2.CAP_PROP_FPS)
        if original_frame_rate == 0:
            raise ValueError(f"Failed to get frame rate for video file: {input_path}")

        frame_interval = int(original_frame_rate // frame_rate)

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, frame_rate, resize_dim, not grayscale)

        frame_count = 0
        prev_gray = None

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            if frame_count % frame_interval == 0:
                # Resize frame
                frame = cv2.resize(frame, resize_dim, interpolation=cv2.INTER_AREA)
                # Convert to grayscale if needed for optical flow
                gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Apply noise reduction
                frame = cv2.fastNlMeansDenoisingColored(frame, None, 10, 10, 7, 21)
                # Apply histogram equalization
                if grayscale:
                    frame = cv2.equalizeHist(frame)
                else:
                    for i in range(3):
                        frame[:, :, i] = cv2.equalizeHist(frame[:, :, i])
                # # Edge detection
                # edges = cv2.Canny(frame, 100, 200)

                # Optical flow calculation
                if prev_gray is not None:
                    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)
                    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
                    hsv = np.zeros_like(frame)
                    hsv[..., 1] = 255
                    hsv[..., 0] = ang * 180 / np.pi / 2
                    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
                    optical_flow = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
                    frame = cv2.addWeighted(frame, 0.5, optical_flow, 0.5, 0)
                prev_gray = gray_frame

                # # Feature extraction using HOG
                # hog = cv2.HOGDescriptor()
                # hog_features = hog.compute(frame)

                # Keypoint descriptors (ORB example)
                orb = cv2.ORB_create()
                kp, des = orb.detectAndCompute(frame, None)
                frame = cv2.drawKeypoints(frame, kp, None, color=(0, 255, 0), flags=0)
                
                out.write(frame)
            frame_count += 1

        cap.release()
        out.release()
        print(f"Successfully processed video: {input_path}")
    except Exception as e:
        print(f"Error processing {input_path}: {e}")

def get_video_files(input_directory):
    video_files = []
    for root, _, files in os.walk(input_directory):
        for file in files:
            if file.endswith(('.mp4', '.avi', '.mov', '.mkv')):
                video_files.append(os.path.join(root, file))
    return video_files

def process_videos(video_files, output_directory, resize_dim, grayscale, frame_rate):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        with tqdm(total=len(video_files), desc="Processing Videos", unit="video") as pbar:
            for video_file in video_files:
                relative_path = os.path.relpath(video_file, input_dir)
                output_file = os.path.join(output_directory, os.path.splitext(relative_path)[0] + '.' + output_format)
                os.makedirs(os.path.dirname(output_file), exist_ok=True)
                future = executor.submit(preprocess_video, video_file, output_file, resize_dim, grayscale, frame_rate)
                futures.append(future)

            for future in futures:
                future.add_done_callback(lambda p: pbar.update())
            for future in futures:
                future.result()  # Wait for all threads to complete

print("Starting video pre-processing...")
video_files = get_video_files(input_dir)
print(f"Found {len(video_files)} video files.")
resize_dim = (resize_width, resize_height)
process_videos(video_files, output_dir, resize_dim, convert_to_grayscale, frame_rate)
print("All videos processed successfully.")


Starting video pre-processing...
Found 100 video files.


Processing Videos:   0%|          | 0/100 [00:00<?, ?video/s]

Processing video: V3C1-100/00120/00120.mp4


[NULL @ 0x7fbc96a26900] missing picture in access unit with size 5
Processing Videos:   0%|          | 0/100 [2:19:37<?, ?video/s]


KeyboardInterrupt: 

# Shot Boundary Detection

In [ ]:
# Configuration
input_dir = 'preprocessed_videos/'
output_dir = 'shot_boundaries/'
keyframe_dir = 'keyframes/'
min_scene_length = 15  # Minimum length of a scene in frames
threshold = 30.0  # Threshold for the ThresholdDetector
min_scene_len = 2  # Minimum number of frames a scene should last
hist_threshold = 0.4  # Threshold for histogram comparison

# Ensure the output and keyframe directories exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(keyframe_dir, exist_ok=True)

def calculate_histogram_difference(frame1, frame2):
    hist1 = cv2.calcHist([frame1], [0], None, [256], [0, 256])
    hist2 = cv2.calcHist([frame2], [0], None, [256], [0, 256])
    cv2.normalize(hist1, hist1)
    cv2.normalize(hist2, hist2)
    return cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)

def detect_shot_boundaries(video_path, output_path, keyframe_path):
    video_manager = VideoManager([video_path])
    scene_manager = SceneManager()

    # Add ContentDetector and ThresholdDetector
    scene_manager.add_detector(ContentDetector(threshold=30.0, min_scene_len=min_scene_length))
    scene_manager.add_detector(ThresholdDetector(threshold=threshold, min_scene_len=min_scene_len))

    video_manager.set_downscale_factor()
    video_manager.start()
    scene_manager.detect_scenes(frame_source=video_manager)
    scenes = scene_manager.get_scene_list()

    # Additional processing for gradual transitions
    cap = cv2.VideoCapture(video_path)
    prev_frame = None
    prev_gray = None
    frame_num = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if prev_frame is not None:
            hist_diff = calculate_histogram_difference(prev_frame, frame)
            if hist_diff < hist_threshold:
                # Gradual transition detected
                scenes.append((frame_num, frame_num + min_scene_len))
            # Motion analysis using optical flow
            if prev_gray is not None:
                flow = cv2.calcOpticalFlowFarneback(prev_gray, gray_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)
                mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
                motion_magnitude = np.mean(mag)
                if motion_magnitude > threshold:
                    scenes.append((frame_num, frame_num + min_scene_len))
        prev_frame = frame
        prev_gray = gray_frame
        frame_num += 1
    cap.release()

    # Remove duplicate and sort scenes
    scenes = sorted(list(set(scenes)))

    # Save shot boundaries to a file
    with open(output_path, 'w') as f:
        for start, end in scenes:
            f.write(f"{start},{end}\n")

    # Extract keyframes for each detected scene
    cap = cv2.VideoCapture(video_path)
    for start, end in scenes:
        cap.set(cv2.CAP_PROP_POS_FRAMES, start)
        ret, frame = cap.read()
        if ret:
            keyframe_filename = os.path.join(keyframe_path, f"{os.path.basename(video_path)}_frame_{start}.jpg")
            cv2.imwrite(keyframe_filename, frame)
    cap.release()

def process_videos(video_files, output_directory, keyframe_directory):
    for video_file in tqdm(video_files, desc="Detecting Shot Boundaries", unit="video"):
        output_file = os.path.join(output_directory, os.path.splitext(os.path.basename(video_file))[0] + '_shots.txt')
        keyframe_path = os.path.join(keyframe_directory, os.path.splitext(os.path.basename(video_file))[0])
        os.makedirs(keyframe_path, exist_ok=True)
        detect_shot_boundaries(video_file, output_file, keyframe_path)

def get_video_files(input_directory):
    video_files = []
    for root, _, files in os.walk(input_directory):
        for file in files:
            if file.endswith(('.mp4', '.avi', '.mov', '.mkv')):
                video_files.append(os.path.join(root, file))
    return video_files

print("Starting shot boundary detection...")
video_files = get_video_files(input_dir)
process_videos(video_files, output_dir, keyframe_dir)
print("Shot boundary detection completed successfully.")


# Feature Extraction

In [ ]:
# Configuration
keyframe_dir = 'keyframes/'
db_name = 'video_features_db'
collection_name = 'features'
batch_size = 32
yolo_model_path = 'yolov5s.pt'  # Using the smallest version of YOLOv5 for demonstration

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize MongoDB client
client = MongoClient('localhost', 27017)
db = client[db_name]
collection = db[collection_name]

# Initialize pre-trained models
vgg_model = VGG16(weights='imagenet', include_top=False, pooling='avg')
resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Load YOLOv5 model
yolo_model = torch.hub.load('ultralytics/yolov5', 'custom', path=yolo_model_path)

def extract_features(model, preprocess_input, img):
    try:
        img = cv2.resize(img, (224, 224))
        img = img.astype('float32')
        img = preprocess_input(img)
        img = np.expand_dims(img, axis=0)
        features = model.predict(img)
        return features.flatten()
    except Exception as e:
        logging.error(f"Error extracting features: {e}")
        return None

def detect_objects_yolo(img):
    try:
        results = yolo_model(img)
        detected_objects = results.pandas().xyxy[0].to_dict(orient="records")
        return detected_objects
    except Exception as e:
        logging.error(f"Error detecting objects with YOLO: {e}")
        return []

def process_keyframes(keyframe_directory, model, preprocess_input, model_name):
    try:
        for root, _, files in os.walk(keyframe_directory):
            for file in tqdm(files, desc=f"Extracting features using {model_name}", unit="frame"):
                if file.endswith('.jpg'):
                    file_path = os.path.join(root, file)
                    img = cv2.imread(file_path)
                    
                    # YOLO Object Detection
                    objects = detect_objects_yolo(img)
                    
                    # CNN Feature Extraction
                    features = extract_features(model, preprocess_input, img)
                    
                    if features is not None:
                        video_id, frame_id = os.path.basename(root), os.path.splitext(file)[0]
                        feature_data = {
                            'video_id': video_id,
                            'frame_id': frame_id,
                            'model': model_name,
                            'features': features.tolist(),
                            'objects': objects
                        }
                        collection.insert_one(feature_data)
    except Exception as e:
        logging.error(f"Error processing keyframes: {e}")

def process_videos(keyframe_directory):
    process_keyframes(keyframe_directory, vgg_model, preprocess_input_vgg, 'VGG16')
    process_keyframes(keyframe_directory, resnet_model, preprocess_input_resnet, 'ResNet50')

logging.info("Starting feature extraction with YOLOv5 integration...")
process_videos(keyframe_dir)
logging.info("Feature extraction with YOLOv5 integration completed successfully.")
